# COVID-19 Prediction Analysis

In [1]:
# Suppress warning messages
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# Import general dependencies
import os 
import pandas as pd
from pprint import pprint
from pathlib import Path
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

In [3]:
#Import connection dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import psycopg2

In [4]:
# Import machine learning dependencies 
import sklearn as skl
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
# from sklearn.metrics import accuracy_score, confusion_matrix
# from imblearn.metrics import classificationn_report_imbalanced 
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

# import keras_tuner as kt

### Data Source

**Our World in Data COVID-19 Dataset Citation**
Hannah Ritchie, Edouard Mathieu, Lucas Rodés-Guirao, Cameron Appel, Charlie Giattino, Esteban Ortiz-Ospina, Joe Hasell, Bobbie Macdonald, Diana Beltekian and Max Roser (2020) - "Coronavirus Pandemic (COVID-19)". Published online at OurWorldInData.org. Retrieved from: [Our World In Data / Coronavirus](https://ourworldindata.org/coronavirus)

In [5]:
# Create connection to SQL database
conn = psycopg2.connect(
    host="finalproject.cjgqtjwgyywe.us-east-2.rds.amazonaws.com",
    database="postgres",
    user="postgres",
    password="FinalProject1")

In [6]:
# Read the OneWorldInData COVID-19 database into a DataFrame
df = pd.read_sql("SELECT * from all_countries_data", conn)
print(df.shape)
df.head()

(187000, 68)


,id_row,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,99331,LBN,Asia,Lebanon,2020-03-01,10.0,6.0,1.286,NaN,NaN,...,26.9,40.7,NaN,2.9,78.93,0.744,NaN,NaN,NaN,NaN
1,31,AFG,Asia,Afghanistan,2020-03-26,80.0,6.0,8.000,2.0,1.0,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,99395,LBN,Asia,Lebanon,2020-05-04,740.0,3.0,4.286,25.0,0.0,...,26.9,40.7,NaN,2.9,78.93,0.744,NaN,NaN,NaN,NaN
3,99426,LBN,Asia,Lebanon,2020-06-04,1306.0,50.0,19.714,28.0,1.0,...,26.9,40.7,NaN,2.9,78.93,0.744,NaN,NaN,NaN,NaN
4,131,AFG,Asia,Afghanistan,2020-07-04,32758.0,348.0,293.714,830.0,7.0,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [7]:
# Set the id_row column as index and reorder records by id.
df = df.set_index('id_row').sort_values(by='id_row')
print(df.shape)
df.head()

(187000, 67)


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
id_row,,,,,,,,,,,,,,,,,,,,,
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [8]:
# Check non-null counts and data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187000 entries, 0 to 198845
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    187000 non-null  object 
 1   continent                                   187000 non-null  object 
 2   location                                    187000 non-null  object 
 3   date                                        187000 non-null  object 
 4   total_cases                                 179392 non-null  float64
 5   new_cases                                   179111 non-null  float64
 6   new_cases_smoothed                          177999 non-null  float64
 7   total_deaths                                160966 non-null  float64
 8   new_deaths                                  160731 non-null  float64
 9   new_deaths_smoothed                         159637 non-null  float64
 

In [9]:
# Count the records in the COVID-19 DataFrame.
records = df["location"].count()
print(f"There are {records} location-specific records in the OWID COVID-19 dataset as of July 5th, 2022.")

There are 187000 location-specific records in the OWID COVID-19 dataset as of July 5th, 2022.


In [10]:
# Check the earliest and latest date in the dataset. 
df["date"].sort_values()

id_row
6822      2020-01-01
115431    2020-01-01
6823      2020-01-02
115432    2020-01-02
115433    2020-01-03
             ...    
18933     2022-07-04
50735     2022-07-04
121987    2022-07-04
49879     2022-07-04
198845    2022-07-04
Name: date, Length: 187000, dtype: object

The OWID COVID-19 data entry started on January 1st, 2020 and has been daily updated till July 4th, 2022 when we pulled the dataset from the [OWID/COVID-19-data GitHub Repository](https://github.com/owid/covid-19-data/tree/master/public/data).

In [11]:
# Count the unique locations included in the COVID-19 dataset. 
countries_count = len(df["location"].unique())
print(f"There are {countries_count} unique values in the OWID COVID-19 location column.")

There are 230 unique values in the OWID COVID-19 location column.


In [12]:
# List the locations included in the COVID-19 dataset.
locations_list = df["location"].unique()
print(f"Locations list: {locations_list}")

Locations list: ['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola' 'Anguilla'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan' 'Bolivia'
 'Bonaire Sint Eustatius and Saba' 'Bosnia and Herzegovina' 'Botswana'
 'Brazil' 'British Virgin Islands' 'Brunei' 'Bulgaria' 'Burkina Faso'
 'Burundi' 'Cambodia' 'Cameroon' 'Canada' 'Cape Verde' 'Cayman Islands'
 'Central African Republic' 'Chad' 'Chile' 'China' 'Colombia' 'Comoros'
 'Congo' 'Cook Islands' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba'
 'Curacao' 'Cyprus' 'Czechia' 'Democratic Republic of Congo' 'Denmark'
 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Eswatini'
 'Ethiopia' 'Faeroe Islands' 'Falkland Islands' 'Fiji' 'Finland' 'France'
 'French Polynesia' 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana'
 'Gibraltar' 'Greece' 'Gre